In [ ]:
# BBStudies
import sys
sys.path.append('/Users/pbelanger/ABPLocal/BBStudies')
sys.path.append('/home/phbelang/abp/BBStudies')
import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.InteractionPoint as inp
import BBStudies.Physics.Detuning as tune
import BBStudies.Plotting.BBPlots as bbplt
import BBStudies.Physics.Base as phys
import BBStudies.Physics.Constants as cst

# JOB imports

import json
import pandas as pd
import numpy as np

import xtrack as xt
import xpart as xp
import xobjects as xo

# JOB imports
import importlib
sys.path.append('../../')
main_002 = importlib.import_module('Jobs.002_user_specific_tasks.main')
main = importlib.import_module('Jobs.003_particle_dist_and_track.main')


context = xo.ContextCpu()


line_path = '/Users/pbelanger/ABPLocal/BBStudies/Executables/py-BB/xtrack/test_data/lhc_with_bb/line_and_particle.json'
# line_path = '../xtrack/test_data/lhc_with_bb/line_and_particle.json'

with open(line_path) as f:
    dct = json.load(f)
line = xt.Line.from_dict(dct['line'])
line.particle_ref = xp.Particles.from_dict(dct['particle'])




line.build_tracker()
twiss = line.twiss().to_pandas()

In [ ]:

# BOKEH
import bokeh.plotting as bk
import bokeh.models as bkmod
import bokeh.layouts as bklay
import bokeh.palettes as bkpalettes


# xsuite
import xtrack as xt
import xmask as xm
import xfields as xf
import xpart as xp

# Custom imports
sys.path.append('../004_export_dashboard')
import bokeh_tools as bktools
import Presets as bkpresets

# BBStudies
import sys
sys.path.append('/Users/pbelanger/ABPLocal/BBStudies')
sys.path.append('../../../../')
import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.InteractionPoint as inp
import BBStudies.Physics.Detuning as tune
import BBStudies.Plotting.BBPlots as bbplt
import BBStudies.Physics.Base as phys
import BBStudies.Physics.Constants as cst



# Setting default values
#------------------------------------------------
_default_fig_width  = 1500
_default_fig_height = 400
_default_fig_pad    = 100



# Importing Collider and Twiss
#-------------------------------------
# collider = xt.Multiline.from_json('../001_configure_collider/zfruits/collider_001.json')
collider = xt.Multiline.from_json('../001_configure_collider/zfruits/collider_BUNCH_0224.json')
twiss = {}
twiss['lhcb1'] = collider['lhcb1'].twiss().to_pandas()
twiss['lhcb2'] = collider['lhcb2'].twiss().reverse().to_pandas()
#-------------------------------------


# Importing Collider and Twiss
#-------------------------------------
collider2 = xt.Multiline.from_json('../001_configure_collider/zfruits/collider_001.json')
twiss2 = {}
twiss2['lhcb1'] = collider2['lhcb1'].twiss().to_pandas()
twiss2['lhcb2'] = collider2['lhcb2'].twiss().reverse().to_pandas()
#-------------------------------------

In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
seq = 'lhcb1'
plt.figure(figsize=(20,10))
plt.plot(twiss[seq].s,twiss[seq].y)
plt.plot(twiss2[seq].s,twiss2[seq].y)

# plt.xlim([4500,6500])


In [ ]:
sequence = 'lhcb1'
bblr_info   = bktools.extract_bblr_info(collider[sequence],twiss[sequence])
bblr_info2   = bktools.extract_bblr_info(collider2[sequence],twiss2[sequence])

In [ ]:
bblr_info

In [ ]:
collider2[sequence].elements

In [ ]:
collider[sequence].elements

In [ ]:
plt.figure()
plt.plot(bblr_info.s,bblr_info.strength_x)
plt.plot(bblr_info2.s,bblr_info2.strength_x)
plt.xlim([6000,7000])

In [ ]:
#-------------------------------------
light_twiss = {}
for sequence in ['lhcb1','lhcb2']:
    light_twiss[sequence] = xPlus.filter_twiss(twiss[sequence].set_index('name'),entries=['drift','..','_entry','_exit']).reset_index()
#-------------------------------------

#-------------------------------------
light_twiss2 = {}
for sequence in ['lhcb1','lhcb2']:
    light_twiss2[sequence] = xPlus.filter_twiss(twiss2[sequence].set_index('name'),entries=['drift','..','_entry','_exit']).reset_index()
#-------------------------------------

In [ ]:
sum(light_twiss2['lhcb1'].name == light_twiss['lhcb1'].name)

In [ ]:
light_twiss['lhcb1']

In [ ]:

n_parts = int(20e2)
nemitt_x = 1.8e-6
nemitt_y = 1.8e-6
sigma_z = 0.09
monitor_at = 'tcp.c6l7.b1'

particles,nemitt_zeta = main.generate_realistic_particles(   n_part      = n_parts,
                                                            r_sig_max   = 3,
                                                            nemitt_x    = nemitt_x,
                                                            nemitt_y    = nemitt_y,
                                                            sigma_z     = sigma_z,
                                                            line        = line,
                                                            at_element  = monitor_at,
                                                            _context    = context)




In [ ]:
coord = particles.to_pandas()


import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,4,figsize=(15,5))

for _ax,xy in zip(ax,[('x','px'),('y','py'),('x','y'),('zeta','delta')]):
    plt.sca(_ax)
    #group = data.data.groupby('BUNCH').get_group(bunch)
    _x,_y = xy
    # _surv = survived[bunch]
    plt.plot(coord.loc[:,_x],coord.loc[:,_y],'.',color='C2',alpha=0.1)
    # plt.legend()
    plt.xlabel(_x)
    plt.ylabel(_y)
    # plt.axis('equal')

In [ ]:
bunch_intensity = 1.4e10
n_part   = int(20e5)
nemitt_x = 1.8e-6
nemitt_y = 1.8e-6
sigma_z  = 0.09

np.random.seed(0)
particles = xp.generate_matched_gaussian_bunch(
         num_particles=n_part, total_intensity_particles=bunch_intensity,
         nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_z=sigma_z,
         line=line)


df    = particles.to_pandas()
sig_x = np.sqrt(twiss.loc[0,'betx']*nemitt_x/particles.gamma0.max())
sig_y = np.sqrt(twiss.loc[0,'bety']*nemitt_y/particles.gamma0.max())



df = df[(df.x/sig_x)**2 + (df.y/sig_y)**2 > 2**2]

In [ ]:
df = particles.to_pandas()
import matplotlib.pyplot as plt

plt.figure()
plt.plot(df.x, df.y, '.',alpha=0.3)
plt.axis('equal')

plt.figure()
plt.plot(df.x, df.px, '.',alpha=0.3)

plt.figure()
plt.plot(df.y, df.py, '.',alpha=0.3)

plt.figure()
plt.plot(df.zeta, df.delta, '.',alpha=0.3)


# plt.figure()
# plt.plot(particles.zeta, particles.ptau/particles.beta0, '.',alpha=0.3)
# 
# self._df['ptau']/self._df['beta0']

In [ ]:
n_part=20000
r_sig_max = 4
np.random.seed(0)
x_in_sigmas, px_in_sigmas = xp.generate_2D_gaussian(n_part*10000)
y_in_sigmas, py_in_sigmas = xp.generate_2D_gaussian(n_part*10000)

filtered_x = ((x_in_sigmas**2 + px_in_sigmas**2) > r_sig_max**2)
filtered_y = ((y_in_sigmas**2 + py_in_sigmas**2) > r_sig_max**2)

filtered = filtered_x | filtered_y


# filtered_x_out = ((x_in_sigmas**2 + px_in_sigmas**2) < r_sig_max**2)
# filtered_y_out = ((y_in_sigmas**2 + py_in_sigmas**2) < r_sig_max**2)

# filtered = np.invert(filtered_x_out | filtered_y_out)

# x_in_sigmas, px_in_sigmas = xp.generate_2D_gaussian(n_part*100)
# y_in_sigmas, py_in_sigmas = xp.generate_2D_gaussian(n_part*100)
# filtered = ((x_in_sigmas**2 + y_in_sigmas**2) > r_sig_max**2)

plt.figure()
plt.plot(x_in_sigmas[filtered], px_in_sigmas[filtered], '.',alpha=0.1)
plt.axis('equal')
plt.xlabel('x')
plt.ylabel('px')



plt.figure()
plt.plot(y_in_sigmas[filtered], py_in_sigmas[filtered], '.',alpha=0.1)
plt.axis('equal')
plt.xlabel('y')
plt.ylabel('py')

plt.figure()
plt.plot(x_in_sigmas[filtered], y_in_sigmas[filtered], '.',alpha=0.1)
plt.axis('equal')
plt.xlabel('x')
plt.ylabel('y')


---
# Option2
---

In [ ]:
bunch_intensity = 1.4e10
n_part   = int(20e5)
nemitt_x = 1.8e-6
nemitt_y = 1.8e-6
sigma_z  = 0.09


# Horizontal plane: generate gaussian distribution in normalized coordinates
np.random.seed(0)
x_in_sigmas, px_in_sigmas = xp.generate_2D_gaussian(n_part)
y_in_sigmas, py_in_sigmas = xp.generate_2D_gaussian(n_part)

filtered = x_in_sigmas**2 + y_in_sigmas**2 > 3**2
x_in_sigmas  = x_in_sigmas[filtered]
px_in_sigmas = px_in_sigmas[filtered]
y_in_sigmas  = y_in_sigmas[filtered]
py_in_sigmas = py_in_sigmas[filtered]

# Longitudinal plane: generate gaussian distribution matched to bucket 
zeta, delta,matcher = xp.generate_longitudinal_coordinates(num_particles=sum(filtered), distribution='gaussian',sigma_z=sigma_z, line=line,return_matcher=True)

# Build particles:
#    - scale with given emittances
#    - transform to physical coordinates (using 1-turn matrix)
#    - handle dispersion
#    - center around the closed orbit
particles = xp.build_particles(line=line,
            zeta=zeta, delta=delta,
            x_norm=x_in_sigmas, px_norm=px_in_sigmas,
            y_norm=y_in_sigmas, py_norm=py_in_sigmas,
            nemitt_x=nemitt_x, nemitt_y=nemitt_y,at_element='tcp.c6l7.b1')

In [ ]:
matcher._compute_emittance(matcher.rfbucket,matcher.psi)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(particles.x, particles.y, '.',alpha=0.3)
# plt.plot(df.x, df.y, '.',alpha=0.3)
plt.axis('equal')

plt.figure()
plt.plot(particles.x, particles.px, '.',alpha=0.3)
# plt.plot(df.x, df.px, '.',alpha=0.3)

plt.figure()
plt.plot(particles.y, particles.py, '.',alpha=0.3)
# plt.plot(df.y, df.py, '.',alpha=0.3)

plt.figure()
plt.plot(particles.zeta, particles.delta, '.',alpha=0.3)
# plt.plot(df.zeta, df.delta, '.',alpha=0.3)


# plt.figure()
# plt.plot(particles.zeta, particles.ptau/particles.beta0, '.',alpha=0.3)
# 
# self._df['ptau']/self._df['beta0']

In [ ]:
x,y = df.zeta, df.delta

x_collx = np.linspace(-0.5,0.5,100)
skew_angle = 127.5 #+ 90
coll_s = 0.1
y_fun_skew = lambda _x: np.tan(np.deg2rad(skew_angle))*_x + coll_s/np.cos(np.deg2rad(180-skew_angle))



theta = np.deg2rad(skew_angle-90)
x_rot = x*np.cos(theta) - y*np.sin(theta)
y_rot = x*np.sin(theta) + y*np.cos(theta)


theta_unskew = -theta
x_unskew = x_rot*np.cos(theta_unskew) - y_rot*np.sin(theta_unskew)
y_unskew = x_rot*np.sin(theta_unskew) + y_rot*np.cos(theta_unskew)




plt.figure()
# plt.plot(x,y,'.',alpha=0.3)
plt.plot(x_rot,y_rot,'.',alpha=0.3)

plt.plot(x_unskew,y_unskew,'.',alpha=0.3,color='C2')
plt.plot(x_collx,y_fun_skew(x_collx),'-',color='k')
plt.axhline(0,color='k')
plt.axis('equal')
plt.xlim(-0.3,0.3)
plt.ylim(-0.3,0.3)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
emitt = 1.8e-6/float(line.particle_ref.gamma0)
# sig = np.sqrt(emitt*twiss.betx[0])
np.sqrt(emitt*twiss.betx[0])
np.sqrt(emitt*twiss.bety[0])

In [ ]:
np.matmul(twiss.W_matrix[0],np.array([1*np.sqrt(emitt),0,0,0,0,0]))[0]

In [ ]:
np.matmul(twiss.W_matrix[0],np.array([0,0,1*np.sqrt(emitt),0,0,0]))[2]

In [ ]:
twiss.W_matrix[0][0,0]*np.sqrt(emitt)

In [ ]:

x = np.array([0,1])
y = np.array([0,0])

x_coll = np.array([1,1])
y_coll = np.array([-10,10])

theta = np.deg2rad(127.5)
x_rot = x*np.cos(theta) - y*np.sin(theta)
y_rot = x*np.sin(theta) + y*np.cos(theta)

x_unskew = x_rot*np.cos(-theta) - y_rot*np.sin(-theta)
y_unskew = x_rot*np.sin(-theta) + y_rot*np.cos(-theta)


x_coll_rot = x_coll*np.cos(theta) - y_coll*np.sin(theta)
y_coll_rot = x_coll*np.sin(theta) + y_coll*np.cos(theta)

plt.figure()

plt.axhline(0,color='k',alpha=0.3)
plt.axvline(0,color='k',alpha=0.3)
# plt.plot(x,y,'-o')
plt.plot(x_rot,y_rot,'-o')
plt.plot(x_coll_rot,y_coll_rot,'-o',color='red')
plt.plot(x_unskew,y_unskew,'-o',color='C2')
plt.axis('equal')
plt.xlim(-2,2)
plt.ylim(-2,2)

